# 01: Data Loading and Cleaning

This notebook loads and cleans the VARBX and benchmark data.


In [1]:
import sys
from pathlib import Path

# Add src to path - robust path resolution for notebooks
# In Jupyter, cwd is usually the project root, not notebooks/
current_dir = Path.cwd()

# Check if we're in notebooks/ directory or at project root
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'config.yml').exists():
    # We're already at project root
    project_root = current_dir
else:
    # Try to find project root by looking for config.yml
    project_root = current_dir
    while project_root != project_root.parent:
        if (project_root / 'config.yml').exists():
            break
        project_root = project_root.parent

sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from src.data.loaders import load_varbx_data, load_sp500_data, load_hfri_ed_data
from src.data.preprocess import align_timeframes, merge_returns_dataframes, clean_returns_dataframe
from src.utils.paths import get_data_interim_path, ensure_dir

# Verify path resolution (you can remove this after confirming it works)
print(f"Project root: {project_root}")
print(f"Config exists: {(project_root / 'config.yml').exists()}")


Project root: /Users/paulgarofalo/Desktop/varbxaltinvfinalproj
Config exists: True


## Load Data


In [2]:
# Load VARBX data
varbx_df = load_varbx_data()
print(f"VARBX data: {len(varbx_df)} observations")
print(varbx_df.head())


VARBX data: 306 observations
        date  return
0 2000-04-30  0.0130
1 2000-05-31  0.0497
2 2000-06-30  0.0770
3 2000-07-31  0.0630
4 2000-08-31  0.0128


In [3]:
# Load benchmark data
sp500_df = load_sp500_data()
hfri_ed_df = load_hfri_ed_data()

print(f"S&P 500 data: {len(sp500_df)} observations")
print(f"HFRI ED (Merger Arbitrage Index) data: {len(hfri_ed_df)} observations")


S&P 500 data: 70 observations
HFRI ED (Merger Arbitrage Index) data: 306 observations


## Clean and Align Data


In [4]:
# Clean each dataset
varbx_clean = clean_returns_dataframe(varbx_df, date_column="date", return_column="return")
sp500_clean = clean_returns_dataframe(sp500_df, date_column="date", return_column="return")
hfri_ed_clean = clean_returns_dataframe(hfri_ed_df, date_column="date", return_column="return")

# Align all datasets to start at April 2000 (2000-04-30)
# All loaders should already filter to this date, but ensure alignment
target_start_date = pd.Timestamp("2000-04-30")
varbx_clean = varbx_clean[varbx_clean["date"] >= target_start_date].copy().reset_index(drop=True)
sp500_clean = sp500_clean[sp500_clean["date"] >= target_start_date].copy().reset_index(drop=True)
hfri_ed_clean = hfri_ed_clean[hfri_ed_clean["date"] >= target_start_date].copy().reset_index(drop=True)

print(f"Aligned all data to start at {target_start_date}")
print(f"VARBX: {len(varbx_clean)} observations, date range: {varbx_clean['date'].min()} to {varbx_clean['date'].max()}")
print(f"S&P 500: {len(sp500_clean)} observations, date range: {sp500_clean['date'].min()} to {sp500_clean['date'].max()}")
print(f"HFRI ED: {len(hfri_ed_clean)} observations, date range: {hfri_ed_clean['date'].min()} to {hfri_ed_clean['date'].max()}")


Aligned all data to start at 2000-04-30 00:00:00
VARBX: 306 observations, date range: 2000-04-30 00:00:00 to 2025-09-30 00:00:00
S&P 500: 70 observations, date range: 2020-02-29 00:00:00 to 2025-11-30 00:00:00
HFRI ED: 306 observations, date range: 2000-04-30 00:00:00 to 2025-09-30 00:00:00


In [5]:
# Merge all returns into single DataFrame
returns_df = merge_returns_dataframes(
    varbx_clean,
    sp500_clean,
    hfri_ed_clean,
    date_column="date",
    suffixes=["varbx", "sp500", "hfri_ed"]
)

print(f"Merged data: {len(returns_df)} observations")
print(f"Date range: {returns_df['date'].min()} to {returns_df['date'].max()}")
print(returns_df.head())


Merged data: 308 observations
Date range: 2000-04-30 00:00:00 to 2025-11-30 00:00:00
        date  return_varbx  return_sp500  return_hfri_ed
0 2000-04-30        0.0130           NaN          0.0247
1 2000-05-31        0.0497           NaN          0.0151
2 2000-06-30        0.0770           NaN          0.0158
3 2000-07-31        0.0630           NaN          0.0119
4 2000-08-31        0.0128           NaN          0.0134


## Save Cleaned Data


In [6]:
# Save to interim directory
interim_path = ensure_dir(get_data_interim_path())
returns_df.to_csv(interim_path / "returns_merged.csv", index=False)
print(f"Saved cleaned data to {interim_path / 'returns_merged.csv'}")


Saved cleaned data to /Users/paulgarofalo/Desktop/varbxaltinvfinalproj/data/interim/returns_merged.csv
